In [3]:
import os
import pandas as pd
import numpy as np
import pickle as pkl

/tmp/ipykernel_26952/3601469004.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
def sort_periodogram(per):
    local_maxima = np.where((per[1:-1] > per[:-2]) & (per[1:-1] > per[2:]))[0] + 1
    idx_highest = local_maxima[np.argsort(per[local_maxima])[::-1]]
    return idx_highest

In [7]:
def top_frequencies(top, idx_highest, freq):
    frequencies = [freq[i] for i in idx_highest]
    return frequencies[:top]

In [8]:
# tamaño max-1 + max-2
def generator_multiples(max):
    mult = np.concatenate([np.arange(1, max, 1), # multiples
                                1./np.arange(2, max, 1)]) # submultiples
    return mult

In [9]:
def relative_error(observed, actual):
    return np.absolute((observed - actual)/actual)

In [10]:
def matrix_band(frequencies_ranking, max, top, period, tol,multiples):
    matrix = [ [ [] for i in range(2*max-3) ] for j in range(top) ]
    for i in range(len(frequencies_ranking)):
        for j in range(len(multiples)):
            error = relative_error(1/(frequencies_ranking[i]*multiples[j]), period)
            tol_bool = error<tol
            matrix[i][j] = [error, tol_bool]
    return matrix

In [11]:
directory = '../dataset'
max_mult = 5
top = 10
tol = 1e-2
multiples = generator_multiples(max_mult)
data_global = pd.read_pickle('../dataset/data_global.pkl')
for folder_type in os.listdir(directory):
    # d_folder_type = 
    d_folder_type = os.path.join(directory, folder_type)
    # folder_type = 'ECL' 'RR'
    if folder_type.find('.') == -1:
        freq = pd.read_pickle('../dataset/'+folder_type+'/frequencies_'+folder_type+'.pkl')
        #lc_folder = '530580438478016768'
        for lc_folder in os.listdir(d_folder_type):
            # porque esta frequencies_ECL.pkl
            if lc_folder.find('.') == -1:
                # d_lc_folder
                dict_matrix = {}
                dict_matrix['multiples'] = multiples
                period = data_global[data_global.index == int(lc_folder)].values[0][0]
                d_lc_folder = os.path.join(d_folder_type,lc_folder)
                d_periodograms = os.path.join(d_lc_folder, 'periodograms.pkl')
                dict_periodograms = pd.read_pickle(d_periodograms)
                for band in dict_periodograms.keys():
                    per = dict_periodograms[band]
                    idx_highest = sort_periodogram(per)
                    freq_ranking = top_frequencies(top, idx_highest,freq)
                    matrix = matrix_band(freq_ranking,max_mult,top,period,tol, multiples)
                    dict_matrix[band] = [freq_ranking, matrix]
                with open(d_lc_folder+'/error_matrixes'+lc_folder+'.pkl', 'wb') as handle:
                    pkl.dump(dict_matrix, handle, protocol=pkl.HIGHEST_PROTOCOL)

        


In [12]:
def print_matrix(dict_matrix):
    print('Multiples: ', dict_matrix['multiples'])
    

In [24]:
def print_matrix(matrix, top, max, freq_ranking):
    for i in range(top):
        print(f'freq ranking {i+1}: {freq_ranking[i]:0.4f}', end='  ')
        for j in range(2*max-3):
            
            print(f"[{matrix[i][j][0]:0.4f}, {matrix[i][j][1]}]", end=" ")
        print()

In [29]:
name = '184457373509254272'
dict_matrix = pd.read_pickle('../dataset/ECL/'+name+'/error_matrixes'+name+'.pkl')
print('Multiples: ', dict_matrix['multiples'])
for band in ['BP', 'G']:
    if band != 'multiples':
        print('Band: '+band)
        matrix = dict_matrix[band][1]
        freq_ranking = dict_matrix[band][0]
        print_matrix(matrix,top,max_mult,freq_ranking)


Multiples:  [1.         2.         3.         4.         0.5        0.33333333
 0.25      ]
Band: BP
freq ranking 1: 1.2902  [0.7725, False] [0.8862, False] [0.9242, False] [0.9431, False] [0.5450, False] [0.3175, False] [0.0900, False] 
freq ranking 2: 0.7261  [0.5958, False] [0.7979, False] [0.8653, False] [0.8989, False] [0.1915, False] [0.2127, False] [0.6170, False] 
freq ranking 3: 0.3116  [0.0580, False] [0.5290, False] [0.6860, False] [0.7645, False] [0.8841, False] [1.8261, False] [2.7682, False] 
freq ranking 4: 1.0533  [0.7213, False] [0.8607, False] [0.9071, False] [0.9303, False] [0.4427, False] [0.1640, False] [0.1147, False] 
freq ranking 5: 0.3701  [0.2069, False] [0.6034, False] [0.7356, False] [0.8017, False] [0.5863, False] [1.3794, False] [2.1725, False] 
freq ranking 6: 1.9733  [0.8513, False] [0.9256, False] [0.9504, False] [0.9628, False] [0.7025, False] [0.5538, False] [0.4050, False] 
freq ranking 7: 0.8809  [0.6668, False] [0.8334, False] [0.8889, False] [0.91

In [30]:
for band in ['RP', 'multiband']:
    if band != 'multiples':
        print('Band: '+band)
        matrix = dict_matrix[band][1]
        freq_ranking = dict_matrix[band][0]
        print_matrix(matrix,top,max_mult,freq_ranking)

Band: RP
freq ranking 1: 1.2902  [0.7725, False] [0.8862, False] [0.9242, False] [0.9431, False] [0.5450, False] [0.3175, False] [0.0900, False] 
freq ranking 2: 0.7261  [0.5958, False] [0.7979, False] [0.8653, False] [0.8989, False] [0.1915, False] [0.2127, False] [0.6170, False] 
freq ranking 3: 1.0533  [0.7213, False] [0.8607, False] [0.9071, False] [0.9303, False] [0.4427, False] [0.1640, False] [0.1147, False] 
freq ranking 4: 0.3116  [0.0580, False] [0.5290, False] [0.6860, False] [0.7645, False] [0.8841, False] [1.8261, False] [2.7682, False] 
freq ranking 5: 1.9732  [0.8512, False] [0.9256, False] [0.9504, False] [0.9628, False] [0.7025, False] [0.5537, False] [0.4050, False] 
freq ranking 6: 0.3700  [0.2067, False] [0.6033, False] [0.7356, False] [0.8017, False] [0.5867, False] [1.3800, False] [2.1734, False] 
freq ranking 7: 0.8809  [0.6668, False] [0.8334, False] [0.8889, False] [0.9167, False] [0.3336, False] [0.0004, True] [0.3328, False] 
freq ranking 8: 1.6392  [0.8209, 

In [40]:
dict_matrix = {'multiples': [],
    'BP': [freq_ranking, matrix],
    'G': [freq_ranking, matrix],
    'RG': [freq_ranking, matrix],
    'multiband': [freq_ranking, matrix]
}